<a href="https://colab.research.google.com/github/parksanghan/machine_learning_Deep_learning/blob/main/CamGear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install keras_cv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 36.5 MB/s eta 0:00:00


In [ ]:
pip install vidgear

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.0/122.0 kB 7.3 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import keras_cv
import cv2
from vidgear.gears import CamGear
import numpy as np
import matplotlib.pyplot as plt

physical_devices = tf.config.list_physical_devices("GPU")
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
  print("GPU memory growth enabled!")
  print("physical device ", physical_devices)
  device_details = tf.config.experimental.get_device_details(physical_devices[0])
  print("device details ", device_details)
except:
  print("Invalid device or cannot modify virtual devices once initialized!")
  pass

GPU memory growth enabled!
physical device  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
device details  {'compute_capability': (7, 5), 'device_name': 'Tesla T4'}


In [ ]:

import google.colab.patches as patches
from google.colab.patches import cv2_imshow
pascal_class_ids = [
    "Aeroplane",
    "Bicycle",
    "Bird",
    "Boat",
    "Bottle",
    "Bus",
    "Car",
    "Cat",
    "Chair",
    "Cow",
    "Dining Table",
    "Dog",
    "Horse",
    "Motorbike",
    "Person",
    "Potted Plant",
    "Sheep",
    "Sofa",
    "Train",
    "Tvmonitor",
    "Total",
]
pascal_class_mapping = dict(zip(range(len(pascal_class_ids)), pascal_class_ids))

model_prediction_decoder = keras_cv.layers.NonMaxSuppression(
    bounding_box_format = "xywh",
    from_logits = True,
    iou_threshold = 0.3,
    confidence_threshold = 0.7,
)

model = keras_cv.models.YOLOV8Detector.from_preset(
    #"yolo_v8_s_backbone_coco",
    "yolo_v8_m_pascalvoc",
    bounding_box_format = "xywh",
    #prediction_decoder = model_prediction_decoder
)

# resizing = keras_cv.layers.Resizing(
#     640, 640, pad_to_aspect_ratio=True, bounding_box_format="xywh"
# )

def detect(frame):
    frame = cv2.resize(frame, (768, 768), interpolation = cv2.INTER_LINEAR)
    #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    encoded_image = cv2.imencode(".jpg", frame)[1]
    tensor_image = tf.image.decode_jpeg(encoded_image.tobytes(), 3)
    #plt.imshow(tensor_image)
    #print(tensor_image.shape)

    detected = model.predict(np.array([tensor_image]))

    #detected = keras_cv.bounding_box.to_ragged(detected)
    detected_images = keras_cv.visualization.draw_bounding_boxes(
        np.array([frame]),
        bounding_boxes = detected,
        color = (0, 0, 255),
        bounding_box_format = "xywh",
        text_thickness = 1,
        line_thickness = 1,
        font_scale = 0.6,
        class_mapping = pascal_class_mapping,
    )

    return detected_images.astype(np.uint8)[0]

In [ ]:
STREAM_URL = "https://www.youtube.com/watch?v=1EiC9bvVGnk"

stream = CamGear(backend= cv2.CAP_GSTREAMER, source =STREAM_URL, stream_mode = True, logging = True).start()
stream_fps = stream.ytv_metadata["fps"]

cv2.namedWindow("Detect!", cv2.WINDOW_AUTOSIZE)

count = 0
while True:
    frame = stream.read()

    if frame is None:
        print("END!")
        break

    detected_frame = detect(frame)
    #detected_frame = cv2.cvtColor(detected_frame, cv2.COLOR_RGB2BGR)

    cv2_imshow("Detect!", detected_frame)


    # if count > 100:
    #     break
    count += 1

cv2.destroyAllWindows()
stream.stop()

print("STOPPED!")

09:13:00 ::    Helper     ::  DEBUG   :: Selecting `best` resolution for streams.
09:13:00 ::    CamGear    ::  DEBUG   :: Enabling Threaded Queue Mode for the current video source!
09:13:00 ::    CamGear    ::   INFO   :: Setting backend `1800` for this source.


RuntimeError: [CamGear:ERROR] :: Source is invalid, CamGear failed to initialize stream on this source!

In [ ]:
import cv2
from vidgear.gears import CamGear
from google.colab.patches import cv2_imshow
import time

# YouTube 라이브 스트리밍 URL (예시)
STREAM_URL = "https://www.youtube.com/watch?v=1EiC9bvVGnk"

# CamGear로 비디오 스트리밍 시작
stream = CamGear(backend= cv2.CAP_GSTREAMER, source =STREAM_URL, stream_mode = True, logging = True).start()

# 프레임 수 계산
count = 0

try:
    while True:
        # 스트리밍 프레임 읽기
        frame = stream.read()

        if frame is None:
            print("END!")
            break

        # OpenCV는 BGR 형식으로 이미지를 읽기 때문에 RGB 형식으로 변환
        detected_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Google Colab에 이미지 표시
        cv2_imshow(detected_frame)  # OpenCV 이미지 표시

        count += 1
        # 조건에 따라 루프 종료 (예: 100프레임 후 종료)
        if count >= 100:
            break

        # 시간 초과로 인한 루프 종료 조건을 위해 약간의 대기 추가
        time.sleep(0.1)  # FPS에 따라 조정

except KeyboardInterrupt:
    print("Stopped by user.")

# 자원 해제
stream.stop()
print("STOPPED!")


09:13:55 ::    Helper     ::  DEBUG   :: Selecting `best` resolution for streams.
09:13:55 ::    CamGear    ::  DEBUG   :: Enabling Threaded Queue Mode for the current video source!
09:13:55 ::    CamGear    ::   INFO   :: Setting backend `1800` for this source.


RuntimeError: [CamGear:ERROR] :: Source is invalid, CamGear failed to initialize stream on this source!

In [ ]:
# GStreamer 패키지 설치 (gstreamer1.0-ffmpeg 제외)
!apt-get install -y gstreamer1.0-tools gstreamer1.0-plugins-base \
gstreamer1.0-plugins-good gstreamer1.0-plugins-bad gstreamer1.0-plugins-ugly

import cv2
from vidgear.gears import CamGear
from google.colab.patches import cv2_imshow
import time

# YouTube 라이브 스트리밍 URL (예시)
STREAM_URL = "https://www.youtube.com/watch?v=1EiC9bvVGnk"

# GStreamer 스트림을 위한 명령어 생성
gst_pipeline = f"rtspsrc location={STREAM_URL} ! decodebin ! videoconvert ! appsink"

# CamGear로 비디오 스트리밍 시작
stream = CamGear(backend=cv2.CAP_GSTREAMER, source=gst_pipeline, stream_mode=True, logging=True).start()

# 프레임 수 계산
count = 0

try:
    while True:
        # 스트리밍 프레임 읽기
        frame = stream.read()

        if frame is None:
            print("END!")
            break

        # OpenCV는 BGR 형식으로 이미지를 읽기 때문에 RGB 형식으로 변환
        detected_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Google Colab에 이미지 표시
        cv2_imshow(detected_frame)

        count += 1
        # 조건에 따라 루프 종료 (예: 100프레임 후 종료)
        if count >= 100:
            break

        # 시간 초과로 인한 루프 종료 조건을 위해 약간의 대기 추가
        time.sleep(0.1)  # FPS에 따라 조정

except KeyboardInterrupt:
    print("Stopped by user.")

# 자원 해제
stream.stop()
print("STOPPED!")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript glib-networking
  glib-networking-common glib-networking-services gsettings-desktop-schemas gsfonts gstreamer1.0-gl
  gstreamer1.0-x imagemagick-6-common liba52-0.7.4 libaa1 libcdparanoia0 libdca0 libdjvulibre-text
  libdjvulibre21 libdv4 libdvdnav4 libdvdread8 libfaad2 libfftw3-double3 libfluidsynth3
  libfreeaptx0 libgraphene-1.0-0 libgs9 libgs9-common libgssdp-1.2-0 libgstreamer-gl1.0-0
  libgstreamer-plugins-bad1.0-0 libgstreamer-plugins-good1.0-0 libgudev-1.0-0 libgupnp-1.2-1
  libgupnp-igd-1.0-4 libidn12 libijs-0.35 libinstpatch-1.0-2 libjbig2dec0 libjson-glib-1.0-0
  libjson-glib-1.0-common libjxr-tools libjxr0 libkate1 libldacbt-enc2 liblqr-1-0 libltc11
  libmagickcore-6.q16-6 libmagickcore-6.q16-6-extra libmagickwand-6.q16-6 libmjpegutils-2.1-0
  libmodplug1 libm

09:32:37 ::    Helper     ::  DEBUG   :: Selecting `best` resolution for streams.
09:32:37 ::    CamGear    ::  DEBUG   :: Enabling Threaded Queue Mode for the current video source!
09:32:37 ::    CamGear    ::   INFO   :: Setting backend `1800` for this source.


RuntimeError: [CamGear:ERROR] :: Source is invalid, CamGear failed to initialize stream on this source!

In [ ]:
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.6/171.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 101.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 89.2 MB/s eta 0:00:00
